<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/seed(preprocessed_with_label).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load and Preprocess EEG Data


In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load EEG Data (Assuming labeled_eeg_data.csv contains 'label' column + 62 EEG channels)
data = pd.read_csv("/content/labeled_eeg_data.csv")

# Features (X) and Labels (y)
X = data.drop('label', axis=1).values  # EEG features (50910, 62)
y = data['label'].values                # Emotion labels (50910,)

# Encode labels (convert emotions to integers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize EEG data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}, Labels: {len(set(y_encoded))} classes")


Train shape: (476160, 11), Test shape: (119040, 11), Labels: 3 classes


#GAN Model Definition (Generator & Discriminator)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Flatten

latent_dim = 100
n_features = X_train.shape[1]  # 62 channels

def build_generator(latent_dim):
    model = Sequential([
        Dense(128, activation="relu", input_dim=latent_dim),
        Dense(n_features, activation="tanh"),
        Reshape((n_features,))
    ])
    return model

def build_discriminator():
    model = Sequential([
        Flatten(input_shape=(n_features,)),
        Dense(128),
        LeakyReLU(0.2),
        Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Create and compile
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# Combined GAN Model
discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="adam")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#GAN Training Function

In [12]:
def train_gan(X_train, epochs=1000, batch_size=64):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_samples = X_train[idx]

        noise = np.random.randn(half_batch, latent_dim)
        fake_samples = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.randn(batch_size, latent_dim)
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)

        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]:.4f}, D Acc: {d_loss[1]*100:.2f}%, G Loss: {g_loss:.4f}")

train_gan(X_train, epochs=1000, batch_size=64)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/1000, D Loss: nan, D Acc: 42.97%, G Loss: 0.7146
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36m

# Generate Synthetic EEG Data (after GAN training)


In [13]:
def generate_synthetic_samples(n_samples):
    noise = np.random.randn(n_samples, latent_dim)
    generated_samples = generator.predict(noise)
    return generated_samples

# Generate synthetic data for LSTM training (equal size as real training data)
X_synthetic = generate_synthetic_samples(len(X_train))

# Assign fake labels (can be random or one class if for augmentation)
y_synthetic = np.random.choice(y_train, size=len(X_train))

# Combine real + synthetic for final LSTM training set
X_combined = np.vstack((X_train, X_synthetic))
y_combined = np.concatenate((y_train, y_synthetic))

# Reshape for LSTM: (samples, time steps, features) - use 1 time step if flat data
X_combined = X_combined.reshape((X_combined.shape[0], 1, X_combined.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(f"Combined Training Shape (LSTM): {X_combined.shape}, Test Shape: {X_test_reshaped.shape}")


14880/14880 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step
Combined Training Shape (LSTM): (952320, 1, 11), Test Shape: (119040, 1, 11)


#LSTM Model Definition & Training

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define LSTM Classifier
model = Sequential([
    LSTM(64, input_shape=(1, n_features)),
    Dense(32, activation='relu'),
    Dense(len(set(y_encoded)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train LSTM on real + synthetic data
history = model.fit(X_combined, y_combined, epochs=10, batch_size=64, validation_data=(X_test_reshaped, y_test))

# Final Accuracy
_, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Final Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 54s 3ms/step - accuracy: 0.2839 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 2/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.2826 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 3/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.2831 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 4/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 52s 3ms/step - accuracy: 0.2835 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 5/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.2832 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 6/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.2832 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 7/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.2824 - loss: nan - val_accuracy: 0.2840 - val_loss: nan
Epoch 8/10
14880/14880 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.2832 - loss: nan - val_accuracy: 0.2840 